## NMEA Parsing, in Python

In [1]:
import json
import nmea_parser as NMEAParser

---------------------
nmea_parser NOT running as main, probably imported.
---------------------


In [4]:
samples = [
    "$IIRMC,092551,A,1036.145,S,15621.845,W,04.8,317,,10,E,A*0D\r\n",
    "$IIMWV,088,T,14.34,N,A*27\r\n",
    "$IIVWR,148.,L,02.4,N,01.2,M,04.4,K*XX\r\n",
    "$IIVTG,054.7,T,034.4,M,005.5,N,010.2,K,A*XX\r\n",
    "$GPTXT,01,01,02,u-blox ag - www.u-blox.com*50\r\n",
    "$GPRMC,183333.000,A,4047.7034,N,07247.9938,W,0.66,196.21,150912,,,A*7C\r\n",
    "$IIGLL,3739.854,N,12222.812,W,014003,A,A*49\r\n"
]

In [5]:

# akeu = sex_to_dec("12", "34.XX")

for sentence in samples:
    print("Parsing {}".format(sentence))
    try:
        nmea_obj = NMEAParser.parse_nmea_sentence(sentence)
        try:
            print('Parsed Object: {}'.format(json.dumps(nmea_obj, indent=2)))
        except TypeError as type_error:
            print('TypeError: {}'.format(type_error))
            print('Parsed Object (raw): {}'.format(nmea_obj))
    except Exception as ex:
        print("Ooops! {}, {}".format(type(ex), ex))
    print("----------------------------------")


Parsing $IIRMC,092551,A,1036.145,S,15621.845,W,04.8,317,,10,E,A*0D

Parsed Object: {
  "type": "rmc",
  "parsed": {
    "valid": "true",
    "position": {
      "latitude": -10.602416666666667,
      "longitude": -156.36408333333333
    },
    "sog": 4.8,
    "cog": 317.0,
    "declination": 10.0,
    "type": "autonomous"
  }
}
----------------------------------
Parsing $IIMWV,088,T,14.34,N,A*27

Ooops! <class 'nmea_parser.NoParserException'>, No parser exists (yet) for MWV
----------------------------------
Parsing $IIVWR,148.,L,02.4,N,01.2,M,04.4,K*XX

Ooops! <class 'nmea_parser.InvalidChecksumException'>, Invalid checksum for $IIVWR,148.,L,02.4,N,01.2,M,04.4,K*XX
----------------------------------
Parsing $IIVTG,054.7,T,034.4,M,005.5,N,010.2,K,A*XX

Ooops! <class 'nmea_parser.InvalidChecksumException'>, Invalid checksum for $IIVTG,054.7,T,034.4,M,005.5,N,010.2,K,A*XX
----------------------------------
Parsing $GPTXT,01,01,02,u-blox ag - www.u-blox.com*50

Parsed Object: {
  "type": 

## Full Sample, RMC Parsing

In [1]:
rmc_data = "$GPRMC,183333.000,A,4047.7034,N,07247.9938,W,0.66,196.21,150912,,,A*7C\r\n"

### Validation

In [2]:
valid = True

In [3]:
if rmc_data[0] != '$':
    valid = False
    print("String does not begin with '$', not valid")
else:
    print("Start of String OK, moving on.")

Start of String OK, moving on.


In [4]:
if valid:
    if rmc_data[-2:] != "\r\n":
        valid = False
        print("Bad string termination")
    else:
        print("String termination OK, moving on.")

String termination OK, moving on.


In [5]:
if valid:
    string_to_validate = rmc_data[1:-5]
    checksum = rmc_data[-4:-2]
    print("Data to validate: {} against {}".format(string_to_validate, checksum))

Data to validate: GPRMC,183333.000,A,4047.7034,N,07247.9938,W,0.66,196.21,150912,,,A against 7C


In [6]:
if valid:
    cs = 0
    char_array = list(string_to_validate)
    for c in range(len(string_to_validate)):
        cs = cs ^ ord(char_array[c])
        print ("Char {} (0x{:02x} 0b{}) -> CheckSum now 0x{:02x} 0b{}".format(
            char_array[c], 
            ord(char_array[c]), 
            str(bin(ord(char_array[c])))[2:].rjust(8, '0'), 
            cs,
            str(bin(cs))[2:].rjust(8, '0')))
    original_cs = int(checksum, 16)
    if original_cs != cs:
        valid = False
        print("Invalid Checksum: Found {:02x}, expected {:02x}".format(cs, original_cs))
    else:
        print("Checksum OK, moving on")
    

Char G (0x47 0b01000111) -> CheckSum now 0x47 0b01000111
Char P (0x50 0b01010000) -> CheckSum now 0x17 0b00010111
Char R (0x52 0b01010010) -> CheckSum now 0x45 0b01000101
Char M (0x4d 0b01001101) -> CheckSum now 0x08 0b00001000
Char C (0x43 0b01000011) -> CheckSum now 0x4b 0b01001011
Char , (0x2c 0b00101100) -> CheckSum now 0x67 0b01100111
Char 1 (0x31 0b00110001) -> CheckSum now 0x56 0b01010110
Char 8 (0x38 0b00111000) -> CheckSum now 0x6e 0b01101110
Char 3 (0x33 0b00110011) -> CheckSum now 0x5d 0b01011101
Char 3 (0x33 0b00110011) -> CheckSum now 0x6e 0b01101110
Char 3 (0x33 0b00110011) -> CheckSum now 0x5d 0b01011101
Char 3 (0x33 0b00110011) -> CheckSum now 0x6e 0b01101110
Char . (0x2e 0b00101110) -> CheckSum now 0x40 0b01000000
Char 0 (0x30 0b00110000) -> CheckSum now 0x70 0b01110000
Char 0 (0x30 0b00110000) -> CheckSum now 0x40 0b01000000
Char 0 (0x30 0b00110000) -> CheckSum now 0x70 0b01110000
Char , (0x2c 0b00101100) -> CheckSum now 0x5c 0b01011100
Char A (0x41 0b01000001) -> Che

### Validations OK
Now splitting the data

In [7]:
members = string_to_validate.split(',')
print("We have {} members:".format(len(members)))
for item in members:
    print(item)

We have 13 members:
GPRMC
183333.000
A
4047.7034
N
07247.9938
W
0.66
196.21
150912


A


In [8]:
if len(members[0]) != 5:
    print("Bad length for sentence prefix and ID")
else:
    device_prefix = members[0][0:2]
    sentence_id = members[0][-3:]
    print("Device Prefix is {}, Sentence ID is {}".format(device_prefix, sentence_id))

Device Prefix is GP, Sentence ID is RMC
